In [1]:
import mujoco
from mujoco import mjx
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax import device_put
import numpy as np 
import yaml
from typing import List, Dict, Text, Any, Sequence, Union, Optional
import time
import functools
import copy
from flax import struct
import logging
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".5"


### Brax base and state classes

In [2]:
from jax.tree_util import tree_map

@struct.dataclass
class Base:
  """Base functionality extending all brax types.

  These methods allow for brax types to be operated like arrays/matrices.
  """

  def __add__(self, o: Any) -> Any:
    return tree_map(lambda x, y: x + y, self, o)

  def __sub__(self, o: Any) -> Any:
    return tree_map(lambda x, y: x - y, self, o)

  def __mul__(self, o: Any) -> Any:
    return tree_map(lambda x: x * o, self)

  def __neg__(self) -> Any:
    return tree_map(lambda x: -x, self)

  def __truediv__(self, o: Any) -> Any:
    return tree_map(lambda x: x / o, self)

  def reshape(self, shape: Sequence[int]) -> Any:
    return tree_map(lambda x: x.reshape(shape), self)

  def select(self, o: Any, cond: jax.Array) -> Any:
    return tree_map(lambda x, y: (x.T * cond + y.T * (1 - cond)).T, self, o)

  def slice(self, beg: int, end: int) -> Any:
    return tree_map(lambda x: x[beg:end], self)

  def take(self, i, axis=0) -> Any:
    return tree_map(lambda x: jnp.take(x, i, axis=axis, mode='wrap'), self)

  def concatenate(self, *others: Any, axis: int = 0) -> Any:
    return tree_map(lambda *x: jnp.concatenate(x, axis=axis), self, *others)

  def index_set(
      self, idx: Union[jax.Array, Sequence[jax.Array]], o: Any
  ) -> Any:
    return tree_map(lambda x, y: x.at[idx].set(y), self, o)

  def index_sum(
      self, idx: Union[jax.Array, Sequence[jax.Array]], o: Any
  ) -> Any:
    return tree_map(lambda x, y: x.at[idx].add(y), self, o)

  def vmap(self, in_axes=0, out_axes=0):
    """Returns an object that vmaps each follow-on instance method call."""

    # TODO: i think this is kinda handy, but maybe too clever?

    outer_self = self

    class VmapField:
      """Returns instance method calls as vmapped."""

      def __init__(self, in_axes, out_axes):
        self.in_axes = [in_axes]
        self.out_axes = [out_axes]

      def vmap(self, in_axes=0, out_axes=0):
        self.in_axes.append(in_axes)
        self.out_axes.append(out_axes)
        return self

      def __getattr__(self, attr):
        fun = getattr(outer_self.__class__, attr)
        # load the stack from the bottom up
        vmap_order = reversed(list(zip(self.in_axes, self.out_axes)))
        for in_axes, out_axes in vmap_order:
          fun = vmap(fun, in_axes=in_axes, out_axes=out_axes)
        fun = functools.partial(fun, outer_self)
        return fun

    return VmapField(in_axes, out_axes)

  def tree_replace(
      self, params: Dict[str, Optional[jax.typing.ArrayLike]]
  ) -> 'Base':
    """Creates a new object with parameters set.

    Args:
      params: a dictionary of key value pairs to replace

    Returns:
      data clas with new values

    Example:
      If a system has 3 links, the following code replaces the mass
      of each link in the System:
      >>> sys = sys.tree_replace(
      >>>     {'link.inertia.mass', jp.array([1.0, 1.2, 1.3])})
    """
    new = self
    for k, v in params.items():
      new = _tree_replace(new, k.split('.'), v)
    return new

  @property
  def T(self):  # pylint:disable=invalid-name
    return tree_map(lambda x: x.T, self)

def _tree_replace(
    base: Base,
    attr: Sequence[str],
    val: Optional[jax.typing.ArrayLike],
) -> Base:
  """Sets attributes in a struct.dataclass with values."""
  if not attr:
    return base

  # special case for List attribute
  if len(attr) > 1 and isinstance(getattr(base, attr[0]), list):
    lst = copy.deepcopy(getattr(base, attr[0]))

    for i, g in enumerate(lst):
      if not hasattr(g, attr[1]):
        continue
      v = val if not hasattr(val, '__iter__') else val[i]
      lst[i] = _tree_replace(g, attr[1:], v)

    return base.replace(**{attr[0]: lst})

  if len(attr) == 1:
    return base.replace(**{attr[0]: val})

  return base.replace(
      **{attr[0]: _tree_replace(getattr(base, attr[0]), attr[1:], val)}
  )

@struct.dataclass
class State(Base):
  """A minimal state class (only containing mjx.Data).

  Args:
    pipeline_state: the physics state, mjx.Data
  """

  data: mjx.Data

In [10]:
def load_params(param_path: Text) -> Dict:
    with open(param_path, "rb") as file:
        params = yaml.safe_load(file)
    return params

params = load_params("params/params.yaml")
model = mujoco.MjModel.from_xml_path(params["XML_PATH"])
mjdata = mujoco.MjData(model)
model.opt.solver = mujoco.mjtSolver.mjSOL_NEWTON
model.opt.iterations = 1
model.opt.ls_iterations = 1

mjx_model = mjx.device_put(model)

"""takes state class and ctrl (action) vector, returns next step's state"""
def single_step(state, ctrl):
    data0 = state.data
    data0 = data0.replace(ctrl=ctrl)
    data = mjx.step(mjx_model, data0)
    state = state.replace(data=data)
    return state

def serial_step(vel):
    data = mujoco.MjData(model)
    print(data.qpos)
    data.qvel[0] = 0
    # qvel[0] = vel
    # data = data.replace(qvel=qvel)
    mujoco.mj_step(model, data)
    
    return data.qpos

def serial_step_mjx(vel):
    mjx_data = mjx.make_data(mjx_model)    
    print(mjx_data.qpos)
    qvel = mjx_data.qvel.at[0].set(vel)
    mjx_data = mjx_data.replace(qvel=qvel)
    mjx_data = mjx.step(mjx_model, mjx_data)
    # mjx.forward(model, mjx_data)
    
    return mjx_data.qpos

In [11]:
process_count = jax.process_count()
process_id = jax.process_index()
local_device_count = jax.local_device_count()
local_devices_to_use = local_device_count
print(
    f"Device count: {jax.device_count()}, process count: "
    f"{process_count} (id {process_id}), local device count: "
    f"{local_device_count}, devices to be used count: {local_devices_to_use}")
device_count = local_devices_to_use * process_count

Device count: 1, process count: 1 (id 0), local device count: 1, devices to be used count: 1


In [12]:
n_envs_small = 1
n_envs_large = 128
key = random.PRNGKey(0)
small_ctrl = random.uniform(key, shape=(n_envs_small, mjx_model.nu))
large_ctrl = random.uniform(key, shape=(n_envs_large, mjx_model.nu))
blah = random.uniform(key, shape=(n_envs_large, 1))

def reset(val: int) -> State:
    """Resets the environment to an initial state."""
    data = mjx.make_data(mjx_model)
    # data = data.replace(qvel=jnp.zer(mjx_model.nv, x))
    data = mjx.forward(mjx_model, data)
    return State(data)

large_ctrl.shape

(128, 30)

In [13]:
reset_fn = jax.jit(jax.vmap(reset))
single_batch_step = jax.vmap(single_step)
# returns the state object with a batch axis for each attribute in data (batch_size=n_envs_large)
env_state = reset_fn(blah)
print(env_state.data.qpos.shape)


(128, 74)


In [14]:
jit_single_batch_step = jax.jit(single_batch_step)

### one step no scan inside step function

In [19]:
jit_single_batch_step = jax.jit(single_batch_step)
start_time = time.time()
steps = 100

def f(state ,_):
    return (jit_single_batch_step(state, large_ctrl), None)
env_state, times = jax.lax.scan(f, env_state, (), length=steps)
    
print(f"{steps * n_envs_large} steps completed in {time.time()-start_time} seconds")

12800 steps completed in 79.27197027206421 seconds


In [16]:
start_time = time.time()

jit_single_batch_step(env_state, large_ctrl)
prev = time.time()
print(f"initial execution time: {prev - start_time}")
for _ in range(steps):
    env_state = jit_single_batch_step(env_state, large_ctrl)
    print(f"{time.time()-prev}")
    prev = time.time()
    
print(f"{steps * n_envs_large} steps completed in {time.time()-start_time} seconds")

initial execution time: 91.26726293563843
0.0034570693969726562
0.009163141250610352
0.008644819259643555
0.008731603622436523
0.023305654525756836
0.05629110336303711
0.05693387985229492
0.0573270320892334
0.044460296630859375
0.04165077209472656
0.041831254959106445
0.041758060455322266
0.04244422912597656
0.0410006046295166
0.04403114318847656
0.04128384590148926
0.04202413558959961
0.04035043716430664
0.03888559341430664
0.04278683662414551
0.04221296310424805
0.04272723197937012
0.04271364212036133
0.041632652282714844
0.042551279067993164
0.04153704643249512
0.042264699935913086
0.04131150245666504
0.04225468635559082
0.041396379470825195
0.0425872802734375
0.041309356689453125
0.04262661933898926
0.04222512245178223
0.04236650466918945
0.04199695587158203
0.041817426681518555
0.04174232482910156
0.0419614315032959
0.0424344539642334
0.04121255874633789
0.04190230369567871
0.04196596145629883
0.04254007339477539
0.04127049446105957
0.042615413665771484
0.04200601577758789
0.04281

blahlablahlahl

In [31]:
# this is not working properly..
@jax.vmap
def single_batch_step_for(ctrl):
    mjx_data = mjx.make_data(mjx_model)
    mjx_data = mjx_data.replace(ctrl=ctrl)
    mjx.step(mjx_model, mjx_data)
    return ctrl

def loopfun(iters):
    return jax.lax.fori_loop(0, iters, lambda i, x: single_batch_step_for(x), large_ctrl)
    
jit_loopfun = jit(loopfun)

start = time.time()
print(jit_loopfun(50))
one = time.time()
print(f"one: {one-start}")
print(jit_loopfun(50))
print(f"two: {time.time()-one}")


[[0.6169447  0.7502353  0.3459301  ... 0.51863337 0.58932376 0.55449045]
 [0.47548127 0.7241138  0.3235253  ... 0.24412751 0.3680569  0.7035271 ]
 [0.6749189  0.26419532 0.22431946 ... 0.65006864 0.8245549  0.13341737]
 ...
 [0.2089827  0.32239807 0.6633911  ... 0.35540724 0.5210892  0.6393193 ]
 [0.3341124  0.7145281  0.17795193 ... 0.37987053 0.28285062 0.5811237 ]
 [0.7939482  0.67065954 0.08375716 ... 0.11895394 0.99804735 0.4388287 ]]
one: 46.03626227378845
[[0.6169447  0.7502353  0.3459301  ... 0.51863337 0.58932376 0.55449045]
 [0.47548127 0.7241138  0.3235253  ... 0.24412751 0.3680569  0.7035271 ]
 [0.6749189  0.26419532 0.22431946 ... 0.65006864 0.8245549  0.13341737]
 ...
 [0.2089827  0.32239807 0.6633911  ... 0.35540724 0.5210892  0.6393193 ]
 [0.3341124  0.7145281  0.17795193 ... 0.37987053 0.28285062 0.5811237 ]
 [0.7939482  0.67065954 0.08375716 ... 0.11895394 0.99804735 0.4388287 ]]
two: 0.006846427917480469


# a different approach

The vectorized function takes multiple sequential steps instead of just one

In [25]:
def take_steps(ctrl, steps, mjx_model):
    mjx_data = mjx.make_data(mjx_model)
    mjx_data = mjx_data.replace(ctrl=ctrl)
    def f(data, _):
      return (
          mjx.step(mjx_model, data),
          None,
      )
      
    mjx_data, _ = jax.lax.scan(f, mjx_data, (), steps)
    return mjx_data.qpos

In [26]:
start_time = time.time()
n_envs_small = 1
n_envs_large = 512
steps = 10

batched_steps = vmap(lambda ctrl: take_steps(ctrl, steps, mjx_model), in_axes=0)

jit_batch_step = jit(batched_steps)

batch_end_data = jit_batch_step(small_ctrl)
prev = time.time()
print(f"initial execution time: {prev - start_time}")
def looper():
    prev = time.time()
    for _ in range(5):
        batch_end_data = jit_batch_step(large_ctrl)
        print(f"{time.time()-prev}")
        prev = time.time()
jit_looper = jit(looper)
jit_looper()

initial execution time: 97.75418162345886
35.59422421455383
0.0002770423889160156
0.00016260147094726562
0.000225067138671875
0.00015163421630859375


In [32]:
5120/.00015

34133333.333333336